In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [ ]:
def calcTaxPA(grossSalaryPA, student_finance = False):
    '''calculates tax paid per year based on 
    UK tax bands'''
    taxFreeAllowance = 12500
    if grossSalaryPA <= taxFreeAllowance:
        taxPA = 0
    elif grossSalaryPA <= 50000:
        taxPA = 0.2*(grossSalaryPA - 12500)
        print("You pay: ", taxPA, " in tax per annum")
    elif grossSalaryPA > 50000 and grossSalaryPA <= 150000:
        taxPA = (0.4*(grossSalaryPA - 50000)) + (0.2*(50000 - 12500))
        print("You pay: ", taxPA, " in tax per annum")
    elif grossSalaryPA > 150000:
        taxPA = (0.45*(grossSalaryPA - 150000)) + (0.4*(150000 - 50000)) + (0.2*(50000 - 12500))
        print("You pay: ", taxPA, " in tax per annum")
    taxPA
    return taxPA


def calcNICPA(grossSalaryPA):
    '''calculates national insurance contribution per year'''
    grossSalaryPW = grossSalaryPA/52
    if grossSalaryPW >= 962:
        NICPW = 0.02*(grossSalaryPW-962) + 0.12*(962-166)
    elif grossSalaryPW > 166:
        NICPW = 0.12*(grossSalaryPW - 166)
    else:
        NICPW = 0
    return NICPW*52


def calcStudentFinPA(grossSalaryPA):
    '''calculates student finance per year'''
    grossSalaryPM = grossSalaryPA/12
    thresholdPM = 2143
    if grossSalaryPM > thresholdPM:
        StudentFinPM = 0.09*(grossSalaryPM-thresholdPM)
    else:
        StudentFinPM = 0
    return StudentFinPM*12


def calcNetSalary(grossSalaryPA):
    '''calculates net salary after tax and all other deductions'''
    netSalaryPA = grossSalaryPA - calcTaxPA(grossSalaryPA) - calcNICPA(grossSalaryPA) - calcStudentFinPA(grossSalaryPA)
    return netSalaryPA


def salaryIncrease(salary):
    increaseStep = np.random.normal(7000, 2000)

    return salary+increaseStep

def findInterestRate(paymentDate):
    year = paymentDate.year
    sdate = historicInterest[str(year)]["start"]
    edate = historicInterest[str(year)]["end"]
    delta = edate - sdate

    allDates = [sdate + timedelta(days=i) for i in range(delta.days+1)]

    if paymentDate in allDates:
#         print("yes")
        rate = historicInterest[str(year)]["rate"]
#         print(rate)
    else:
        rate = historicInterest[str(year-1)]["rate"]
#         print("next year", rate)
    return rate


def populateInterestRate(myPayments):
    myPayments["interestRate"] = 0
    myPayments["totalPayment"] = 0
    myPayments["cumulativeTotal"] = 0
    for i, row in myPayments.iterrows():
    #     print(row)
        rate = findInterestRate(myPayments.at[i, "Payment Date"].date())
#         print(rate)
        myPayments.loc[i,"interestRate"] = rate
#         print(rate+1)
        myPayments.loc[i, "totalPayment"] = myPayments.loc[i, "Gross"]*(rate+1)
        myPayments.loc[i, "cumulativeTotal"] = myPayments.loc[i, "Gross"]*(rate+1)
    return myPayments

In [2]:
## build this dictionary from a table which is read in as a csv - potentially even scrape from the student finance website

historicInterest = {"2012": {"start": date(2012, 9, 1), "end": date(2013, 8, 31), "rate": 0.066}, 
                    "2013": {"start": date(2013, 9, 1), "end": date(2014, 8, 31), "rate": 0.063}, 
                    "2014": {"start": date(2014, 9, 1), "end": date(2015, 8, 31), "rate": 0.055}, 
                    "2015": {"start": date(2015, 9, 1), "end": date(2016, 8, 31), "rate": 0.039}, 
                    "2016": {"start": date(2016, 9, 1), "end": date(2017, 8, 31), "rate": 0.046}, 
                    "2017": {"start": date(2017, 9, 1), "end": date(2018, 8, 31), "rate": 0.0618}, 
                    "2018": {"start": date(2018, 9, 1), "end": date(2019, 8, 31), "rate": 0.063}
                   }


# incomeTax = {12500: 0.0, 50000: 0.2, 150000: 0.4, 0.45}
current_wd = os.getcwd()
file_name = "trimmed_data.csv"

def readPayments(file_name, date_header="Payment Date"):
    dataLoc = os.path.join(current_wd, "data", file_name)
    paymentsData = pd.read_csv(dataLoc)
    paymentsData[date_header] = pd.to_datetime(paymentsData[date_header])
    
    return paymentsData

In [3]:
## Read in data

myPayments = pd.read_csv("trimmed_data.csv", sep=",")
myPayments["Payment Date"] = pd.to_datetime(myPayments["Payment Date"])

In [67]:
myPayments = populateInterestRate(myPayments)
graduationAmount = myPayments.sum()["totalPayment"]*1.061
graduationplusOne = graduationAmount*1.063



You pay:  4500.0  in tax per annum


26500.28

# To-do
* Function to calculate student finance owed - need this as separate so can calculate cumulative total over simulated years 